<a href="https://colab.research.google.com/github/oscarvilla/blog_entries_analysis/blob/master/upload_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Inicio
Lo primero es cargar las librerías que vamos a necesitar; de acuerdo con el método presentado en
https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92

In [1]:
import pandas as pd
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [15]:
!pip install langdetect
from langdetect import detect

    100% |████████████████████████████████| 1.0MB 22.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e332867d65e7617ddafe
Successfully built langdetect


## Vamos a importar desde Drive mismo.

In [2]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

    100% |████████████████████████████████| 993kB 20.9MB/s 


Una vez que hemos obtenido el link para compartir el archivo que vamos a importar, lo pegamos aquí (sólo se cambia si cambiamos el archivo)

In [0]:
link = 'https://drive.google.com/open?id=1GjWbMO139LNna2bZbUShmWhPZ06J8hKc' # The shareable link


Dado que lo que se quiere es la parte del id, lo separamos

In [4]:
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

1GjWbMO139LNna2bZbUShmWhPZ06J8hKc


Y para importar el csv como data frame de Pandas

In [0]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('blogtext.csv')  
df = pd.read_csv('blogtext.csv')

Veamos las primeas filas del archivo importado

In [6]:
df[0:6]

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...
5,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",I had an interesting conversation...


Para hacer pruebas, tomamos una submuestra del total.

In [0]:
df2 = df[0:29]

En 'sia' instaciamos el Sentiment Intensity Analyzer, que es al que el iterarador le va a pasar cada texto para que nos devuelva el análisis de la intensidad de sentimientos en él: 
- pos: proporción de positividad del texto [0, 1]
- neu: proporción de neutralidad del texto [0, 1]
- neg: proporción de negatividad del texto [0, 1]
Los tres suman 1
- compound: la combinación de los tres anteriores que arroja un número entre [-1, 1] extremadamente negativo y extremadamente positivo, respectivamente.



In [0]:
sia = SIA()
results = []

for n in df2['text']:
    pol = sia.polarity_scores(n)
    pol['text'] = n
    results.append(pol)

Aquí vemos los resultados. Lo que de paso nos permite observar que hay entradas de texto que no están en inglés (están en holandés). En lo que sigo trato de ver el idioma

In [0]:
df[df]

In [45]:
responses = []

for n in df['text']:
  if detect(n) == 'en':
    res = 1
  else:
    res = 0
  responses.append(res)

LangDetectException: ignored

In [44]:
sum(responses) / len(responses)

0.9655172413793104

In [13]:
results = pd.DataFrame(results)           
results

,compound,neg,neu,pos,text
0,0.0000,0.000,1.000,0.000,"Info has been found (+/- 100 pages,..."
1,0.0000,0.000,1.000,0.000,These are the team members: Drewe...
2,-0.8167,0.090,0.814,0.097,In het kader van kernfusie op aarde...
3,0.0000,0.000,1.000,0.000,testing!!! testing!!!
4,0.8805,0.000,0.841,0.159,Thanks to Yahoo!'s Toolbar I can ...
5,0.9847,0.040,0.874,0.086,I had an interesting conversation...
6,0.8929,0.078,0.787,0.136,Somehow Coca-Cola has a way of su...
7,0.7430,0.073,0.842,0.085,"If anything, Korea is a country o..."
8,-0.8248,0.101,0.804,0.095,Take a read of this news article ...
9,-0.5588,0.099,0.827,0.073,I surf the English news sites a l...


In [0]:
results['label'] = 0 # todos los resultados a cero por defecto (neutro)
results.loc[results['compound'] > 0.2, 'label'] = 1 # positivo si > 0.2
results.loc[results['compound'] < -0.2, 'label'] = -1 # negativo si < -0.2
results

,compound,neg,neu,pos,text,label
0,0.0000,0.000,1.000,0.000,"Info has been found (+/- 100 pages,...",0
1,0.0000,0.000,1.000,0.000,These are the team members: Drewe...,0
2,-0.8167,0.090,0.814,0.097,In het kader van kernfusie op aarde...,-1
3,0.0000,0.000,1.000,0.000,testing!!! testing!!!,0
4,0.8805,0.000,0.841,0.159,Thanks to Yahoo!'s Toolbar I can ...,1
5,0.9847,0.040,0.874,0.086,I had an interesting conversation...,1
6,0.8929,0.078,0.787,0.136,Somehow Coca-Cola has a way of su...,1
7,0.7430,0.073,0.842,0.085,"If anything, Korea is a country o...",1
8,-0.8248,0.101,0.804,0.095,Take a read of this news article ...,-1
9,-0.5588,0.099,0.827,0.073,I surf the English news sites a l...,-1


In [0]:
df = df2.join(results, lsuffix='_orig', rsuffix='_res').drop(labels = ['text_res', 'neg', 'neu', 'pos'], axis = 1)
df

,id,gender,age,topic,sign,date,text_orig,compound,label
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,...",0.0000,0
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...,0.0000,0
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...,-0.8167,-1
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!,0.0000,0
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...,0.8805,1
5,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",I had an interesting conversation...,0.9847,1
6,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",Somehow Coca-Cola has a way of su...,0.8929,1
7,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004","If anything, Korea is a country o...",0.7430,1
8,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",Take a read of this news article ...,-0.8248,-1
9,3581210,male,33,InvestmentBanking,Aquarius,"09,June,2004",I surf the English news sites a l...,-0.5588,-1
